# Initialize Environment

In [ ]:
import matplotlib, matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import numpy as np
from scipy.stats import norm
from scipy.io import loadmat
import glob
import pandas as pd
import datetime
import pandas_datareader.tsp as tsp
import pandas_datareader.data as pdr
import requests_cache
import yfinance as yf
yf.pdr_override() # https://pypi.org/project/yfinance/

from copy import deepcopy
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Seed Random number generator for repeatability
np.random.seed(19690720)

plt.style.use('default')
#matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams.update({'font.size': 18})

# Load Data
## Data Caches

In [ ]:
session = requests_cache.CachedSession(cache_name='cache', 
                                       backend='sqlite', 
                                       expire_after=datetime.timedelta(hours=8))

## Vanguard

In [ ]:
try:
    tickers
except NameError:    
    tickers = [ 'VTSAX', 'VSGAX', 'VGSLX', 'VHYAX', 'VTMSX', 'VTCLX', 'VTMFX', 'VBTLX', 'VWAHX', 'VGSTX', 'VIHAX', 'VWIGX' ]

data = pdr.get_data_yahoo(
        # tickers list or string as well
        tickers = tickers,

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "max",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1d",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = False,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None,
    
        # session cache
        session = session
    )

## TSP
Load TSP data using the pandas_datareader utility. Allow for 8-hour persistent caching. 

In [ ]:
tspreader = tsp.TSPReader(start=datetime.date.today() - datetime.timedelta(days=365*25),
                          session=session)
full_tspdata = tspreader.read()

# write the TSP data, last 90 values, in reversed order
#full_tspdata.iloc[-90:].iloc[::-1].to_excel('tsp_shareprices.xls', sheet_name='TSP Data')

#tspdata = full_tspdata.drop(['L 2025', 'L 2030', 'L 2035', 'L 2040', 'L 2045', 'L 2050', 'L 2055', 'L 2060', 'L 2065'],axis=1)
tspdata = full_tspdata

# Derived Data

In [ ]:
#print(data)
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
ddata = data.swaplevel(0,1,axis=1).drop(['Open', 'High', 'Low', 'Volume'], axis=1)['Close']
#print(type(data), data.columns, ddata.columns)

#print(type(tspdata), tspdata, tspdata.columns)